In [4]:
from google.colab import userdata
import os

# 1. Récupération des secrets
try:
    token = userdata.get('GITHUB_TOKEN')
except ImportError:
    print("Erreur: Assurez-vous d'avoir ajouté le secret GITHUB_TOKEN dans la barre latérale.")
    token = input("Entrez votre token manuellement: ")

# 2. Configuration des variables
username = "Thorbjornen"
repo_owner = "esmeml" # Le compte qui possède le repo
repo_name = "parliament_speaker_diarisation"
user_email = "niels.groeninck4@gmail.com"

# 3. Configuration de l'identité Git (nécessaire pour commit)
!git config --global user.email "{user_email}"
!git config --global user.name "{username}"

# 4. Clonage du dépôt avec authentification
# La syntaxe est : https://<token>@github.com/<owner>/<repo>.git
clone_url = f"https://{token}@github.com/{repo_owner}/{repo_name}.git"

# On clone uniquement si le dossier n'existe pas déjà
if not os.path.exists(repo_name):
    !git clone {clone_url}
    print(f"Dépôt {repo_name} cloné avec succès !")
else:
    print(f"Le dossier {repo_name} existe déjà.")

# 5. Se déplacer dans le dossier du dépôt
%cd {repo_name}

Cloning into 'parliament_speaker_diarisation'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 3), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 1.25 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Dépôt parliament_speaker_diarisation cloné avec succès !
/content/parliament_speaker_diarisation


In [ ]:
from google.colab import drive
import shutil
import os

# accès au drive
drive.mount('/content/drive')

nom_du_notebook = "Notebook_Qui_Parle.ipynb"

chemin_source = f"/content/drive/My Drive/Colab Notebooks/{nom_du_notebook}"

# Nom du dossier Git (celui cloné précédemment)
nom_dossier_git = "parliament_speaker_diarisation"
# -------------------------------

# 2. Vérification que le fichier source existe
if not os.path.exists(chemin_source):
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'endroit : {chemin_source}")
    print("Vérifiez le nom du fichier ou s'il est dans un sous-dossier de votre Drive.")
else:
    destination_dir = f"/content/{nom_dossier_git}"
    os.makedirs(destination_dir, exist_ok=True) # S'assurer que le dossier de destination existe

    # 3. Copie du notebook vers le dossier Git
    destination = os.path.join(destination_dir, nom_du_notebook)
    shutil.copy(chemin_source, destination)
    print(f"Succès : Le notebook a été copié dans le dossier Git !")

    # 4. Envoi vers GitHub
    # Changement du repertoire de travail
    os.chdir(destination_dir)

    print("Envoi vers GitHub en cours...")
    !git add "{nom_du_notebook}"  # On ajoute spécifiquement le notebook
    !git commit -m "Mise à jour du notebook de déploiement"
    !git push origin main # Ou 'master' selon votre branche

    print("Terminé ! Le notebook est maintenant sur le repo.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERREUR : Le dossier '/content/parliament_speaker_diarisation' n'est pas un dépôt Git valide.
Veuillez exécuter la première cellule du notebook pour cloner le dépôt avant de continuer.


In [ ]:
# Installation des librairies nécessaires pour le traitement audio et le XML
!pip install librosa soundfile lxml tensorflow

import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import xml.etree.ElementTree as ET
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive

# 1. Connexion au Google Drive
drive.mount('/content/drive')
print("Google Drive connecté.")

In [ ]:
AUDIO_FILE_NAME = '4_decembre.wav'
XML_FILE_NAME = '4_decembre.xml'

# Chemins (Racine du Drive)
BASE_DIR = '/content/drive/MyDrive'
AUDIO_PATH = os.path.join(BASE_DIR, AUDIO_FILE_NAME)
XML_PATH = os.path.join(BASE_DIR, XML_FILE_NAME)

# Dossier où seront créés les petits segments audio
OUTPUT_DATASET_DIR = os.path.join(BASE_DIR, 'Dataset_Assemblee_CTC')
CSV_MANIFEST_PATH = os.path.join(OUTPUT_DATASET_DIR, 'metadata.csv')

# Paramètres Audio pour CTC
TARGET_SR = 16000  # Standard pour la reconnaissance vocale (16kHz)

# Création du dossier de sortie s'il n'existe pas
os.makedirs(OUTPUT_DATASET_DIR, exist_ok=True)

print(f"Dossier de travail : {OUTPUT_DATASET_DIR}")
if not os.path.exists(AUDIO_PATH):
    print(f"ERREUR : Le fichier audio '{AUDIO_FILE_NAME}' est introuvable à la racine du Drive !")
else:
    print(f"Fichier audio détecté : {AUDIO_PATH}")

In [ ]:
import xml.etree.ElementTree as ET
import json
import os
import re

# Config
XML_FILE = '/content/drive/MyDrive/4_decembre.xml'
JSON_OUTPUT = '/content/drive/MyDrive/segments_clean.json'

def xml_to_json_segments(xml_path):
    print(f"Lecture du fichier : {xml_path}")

    with open(xml_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # On retire les namespaces
    content = re.sub(r' xmlns="[^"]+"', '', content)

    root = ET.fromstring(content)

    raw_segments = []

    # On utilise .iter() pour aller chercher l'info partout
    all_paras = list(root.iter('paragraphe'))
    print(f"Nombre de paragraphes trouvés : {len(all_paras)}")

    for para in all_paras:
        texte_node = para.find('texte')

        if texte_node is not None and 'stime' in texte_node.attrib:
            try:
                start_time = float(texte_node.attrib['stime'])

                # 1. Extraction brute de tout le texte (y compris dans les balises enfants)
                raw_text = "".join(texte_node.itertext())

                # 2. Enlever les \n
                clean_text = " ".join(raw_text.split())

                # 3. Chercher le locuteur
                speaker = "Inconnu"
                orateurs = para.find('orateurs')
                if orateurs is not None:
                    orateur = orateurs.find('orateur')
                    if orateur is not None:
                        nom = orateur.find('nom')
                        if nom is not None:
                            speaker = nom.text

                if clean_text:
                    raw_segments.append({
                        'start': start_time,
                        'speaker': speaker,
                        'text': clean_text
                    })

            except ValueError:
                continue

    # Tri et calcul des durées
    raw_segments.sort(key=lambda x: x['start'])

    final_dataset = []

    for i in range(len(raw_segments) - 1):
        curr = raw_segments[i]
        nxt = raw_segments[i+1]

        duration = nxt['start'] - curr['start']

        # Filtres de qualité
        # On garde si c'est raisonnable (entre 0.5s et 60s)
        # On augmente un peu la tolérance car les applaudissements peuvent être longs
        if 0.5 <= duration <= 60.0:
            curr['end'] = nxt['start']
            curr['duration'] = round(duration, 3)
            final_dataset.append(curr)

    # Sauvegarde
    with open(JSON_OUTPUT, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, indent=4, ensure_ascii=False)

    print(f"Succès ! {len(final_dataset)} segments sauvegardés dans :")
    print(f"   -> {JSON_OUTPUT}")

    # Vérification visuelle
    if len(final_dataset) > 0:
        print("\nExemple corrigé (regardez le champ 'text') :")
        # On cherche un segment qui contient une parenthèse pour vérifier
        for seg in final_dataset:
            if '(' in seg['text']:
                print(json.dumps(seg, indent=4, ensure_ascii=False))
                break

# Lancer la correction
xml_to_json_segments(XML_FILE)

In [ ]:
import json
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

# Config
# Le fichier JSON créé précédemment
JSON_FILE = '/content/drive/MyDrive/segments_clean.json'
# Fichier audio source
AUDIO_FILE = '/content/drive/MyDrive/4_decembre.wav'
# Le dossier pour petits fichiers wav (segments)
OUTPUT_DIR = '/content/drive/MyDrive/Dataset_Assemblee_CTC'

# Création du dossier si nécessaire
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_audio_dataset_from_json():
    """
    Lit le JSON, charge l'audio, découpe et sauvegarde les segments.
    Génère aussi le CSV final pour l'entraînement.
    """

    # 1. Vérification du fichier JSON
    if not os.path.exists(JSON_FILE):
        print(f"ERREUR : Le fichier JSON {JSON_FILE} n'existe pas !")
        print("   -> Avez-vous bien lancé la fonction xml_to_json_segments() avant ?")
        return

    # 2. Chargement des segments depuis le JSON
    print(f"Chargement du JSON : {JSON_FILE}")
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        segments = json.load(f)
    print(f"{len(segments)} segments chargés à traiter.")

    # 3. Chargement de l'audio complet
    print(f"Chargement du fichier audio complet en mémoire (16kHz)... Cela peut prendre 1 à 2 minutes.")
    # On force le chargement en 16000 Hz car c'est le standard pour CTC/ASR
    try:
        y, sr = librosa.load(AUDIO_FILE, sr=16000)
        print(f"Audio chargé ! Durée totale : {len(y)/sr:.2f} secondes.")
    except Exception as e:
        print(f"ERREUR lors du chargement de l'audio : {e}")
        return

    csv_data = []

    print("Découpage et sauvegarde des segments audio...")
    # tqdm pour voir la progression
    for i, seg in enumerate(tqdm(segments)):
        try:
            # Calcul des index de début et fin
            start_sample = int(seg['start'] * sr)
            end_sample = int(seg['end'] * sr)

            # Extraction du bout d'audio
            # Vérification pour ne pas dépasser la taille du fichier
            if end_sample > len(y):
                end_sample = len(y)

            clip = y[start_sample:end_sample]

            # Sauvegarde du petit fichier wav
            filename = f"segment_{i:05d}.wav"
            path = os.path.join(OUTPUT_DIR, filename)
            sf.write(path, clip, sr)

            # On ajoute les infos pour le CSV
            csv_data.append({
                "filename": filename,
                "text": seg['text'],
                "speaker": seg['speaker'],
                "duration": seg['duration']
            })

        except Exception as e:
            print(f"Erreur sur le segment {i} : {e}")
            continue

    # 4. Création du fichier Metadata (CSV)
    df = pd.DataFrame(csv_data)
    csv_path = os.path.join(OUTPUT_DIR, 'metadata.csv')
    df.to_csv(csv_path, index=False)

    print(f"\nTRAITEMENT TERMINÉ")
    print(f"   -> {len(df)} fichiers audio créés dans : {OUTPUT_DATASET_DIR}")
    print(f"   -> Fichier index (CSV) créé : {csv_path}")
    print("\n   Aperçu du CSV :")
    print(df.head())

# Lancement
create_audio_dataset_from_json()

Dans la cellule précédente on a eu un problème de décallage

Connexion au drive

In [ ]:
from google.colab import drive

# 1. Connexion au Google Drive
drive.mount('/content/drive')
print("Google Drive connecté.")

In [ ]:
Whisper

In [ ]:
import json
import os
import soundfile as sf
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# Config
# Fichiers
JSON_FILE = '/content/drive/MyDrive/segments_perfect.json'
AUDIO_FILE = '/content/drive/MyDrive/4_decembre.wav'

# Dossier de sortie
OUTPUT_DIR = '/content/drive/MyDrive/Dataset_Assemblee_Final'
WAVS_DIR = os.path.join(OUTPUT_DIR, 'wavs')

# Paramètres audio cibles pour l'entraînement
TARGET_SR = 16000 # 16kHz est le standard
Target_CHANNELS = 1 # Mono

# Création de l'arborescence
os.makedirs(WAVS_DIR, exist_ok=True)

def cut_and_save_dataset():
    # 1. Vérifications
    if not os.path.exists(JSON_FILE):
        print(f"ERREUR : Le JSON {JSON_FILE} n'existe pas.")
        return

    print(f"Lecture du JSON : {JSON_FILE}")
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        segments = json.load(f)
    print(f"{len(segments)} segments à traiter.")

    # 2. Analyse du fichier audio source
    print("Analyse du fichier audio source...")
    try:
        info = sf.info(AUDIO_FILE)
        native_sr = info.samplerate
        print(f"   -> Source : {info.duration/3600:.2f} heures @ {native_sr}Hz")
    except Exception as e:
        print(f"Erreur lecture audio : {e}")
        return

    csv_data = []

    print("Découpage, Rééchantillonnage (16k) et Sauvegarde...")

    # Barre de progression
    for i, seg in enumerate(tqdm(segments)):
        try:
            # Calcul des bornes
            start_sec = seg['start']
            end_sec = seg['end']
            duration = end_sec - start_sec

            # Sécurité : on ignore les segments vides ou aberrants
            if duration < 0.1: continue

            # Conversion en frames (échantillons) pour soundfile
            start_frame = int(start_sec * native_sr)
            frames_to_read = int(duration * native_sr)

            # Lecture stream
            # On lit seulement le petit morceau nécessaire
            audio_clip, _ = sf.read(AUDIO_FILE, start=start_frame, frames=frames_to_read, always_2d=True)

            # Traitement audio
            # 1. Conversion Stéréo -> Mono (Moyenne des canaux)
            if audio_clip.shape[1] > 1:
                audio_clip = np.mean(audio_clip, axis=1)
            else:
                audio_clip = audio_clip.flatten()

            # 2. Rééchantillonnage vers 16kHz (si nécessaire)
            if native_sr != TARGET_SR:
                # librosa.resample est efficace sur des petits clips
                audio_clip = librosa.resample(audio_clip, orig_sr=native_sr, target_sr=TARGET_SR)

            # Sauvegarde
            filename = f"segment_{i:05d}.wav"
            file_path = os.path.join(WAVS_DIR, filename)

            # On écrit le fichier propre (16k, mono)
            sf.write(file_path, audio_clip, TARGET_SR)

            # Metadata
            # On ajoute les infos pour le CSV final
            csv_data.append({
                "filename": filename,          # Nom du fichier (relatif)
                "text": seg['text'],           # Le texte de Whisper
                "speaker": seg['speaker'],     # L'orateur du XML
                "duration": round(len(audio_clip)/TARGET_SR, 2) # Durée réelle
            })

        except Exception as e:
            print(f"Erreur sur le segment {i} : {e}")
            continue

    # 3. Création du fichier Index (CSV)
    df = pd.DataFrame(csv_data)
    csv_path = os.path.join(OUTPUT_DIR, 'metadata.csv')
    df.to_csv(csv_path, index=False)

    print(f"\nDATASET FINAL TERMINÉ !")
    print(f"Dossier : {OUTPUT_DIR}")
    print(f"CSV Index : {csv_path}")
    print(f"Nombre d'exemples : {len(df)}")
    print("\nAperçu des données :")
    print(df.head())

# Lancement
cut_and_save_dataset()

Envoie sur le repo

In [ ]:
from google.colab import drive
import shutil
import os

# accès au drive
drive.mount('/content/drive')

nom_du_notebook = "Notebook_Qui_Parle.ipynb"

chemin_source = f"/content/drive/My Drive/Colab Notebooks/{nom_du_notebook}"

# Nom du dossier Git (celui cloné précédemment)
nom_dossier_git = "parliament_speaker_diarisation"
# -------------------------------

# 2. Vérification que le fichier source existe
if not os.path.exists(chemin_source):
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'endroit : {chemin_source}")
    print("Vérifiez le nom du fichier ou s'il est dans un sous-dossier de votre Drive.")
else:
    # 3. Copie du notebook vers le dossier Git
    destination = f"/content/{nom_dossier_git}/{nom_du_notebook}"
    shutil.copy(chemin_source, destination)
    print(f"Succès : Le notebook a été copié dans le dossier Git !")

    # 4. Envoi vers GitHub
    # Changement du repertoire de travail
    os.chdir(f"/content/{nom_dossier_git}")

    print("Envoi vers GitHub en cours...")
    !git add "{nom_du_notebook}"  # On ajoute spécifiquement le notebook
    !git commit -m "Mise à jour du notebook de déploiement"
    !git push origin main # Ou 'master' selon votre branche

    print("Terminé ! Le notebook est maintenant sur le repo.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Succès : Le notebook a été copié dans le dossier Git !
Envoi vers GitHub en cours...
[main fdbf76b] Mise à jour du notebook de déploiement
 1 file changed, 1 insertion(+)
 create mode 100644 Notebook_Qui_Parle.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 6.46 KiB | 6.46 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/esmeml/parliament_speaker_diarisation.git
   c854166..fdbf76b  main -> main
Terminé ! Le notebook est maintenant sur le repo.


PARTIE CLUSTERING

In [ ]:
# Installation des librairies nécessaires
# On force des versions compatibles pour éviter les erreurs rouges
!pip install "numpy<2.0" "pandas<3.0" "pyannote.audio==3.1.1" soundfile

In [3]:
from google.colab import drive
import shutil
import os

# accès au drive
drive.mount('/content/drive')

nom_du_notebook = "Notebook_Qui_Parle.ipynb"

chemin_source = f"/content/drive/My Drive/Colab Notebooks/{nom_du_notebook}"

# Nom du dossier Git (celui cloné précédemment)
nom_dossier_git = "parliament_speaker_diarisation"
# -------------------------------

# 2. Vérification que le fichier source existe
if not os.path.exists(chemin_source):
    print(f"ERREUR : Le fichier n'a pas été trouvé à l'endroit : {chemin_source}")
    print("Vérifiez le nom du fichier ou s'il est dans un sous-dossier de votre Drive.")
else:
    # 3. Copie du notebook vers le dossier Git
    destination = f"/content/{nom_dossier_git}/{nom_du_notebook}"
    shutil.copy(chemin_source, destination)
    print(f"Succès : Le notebook a été copié dans le dossier Git !")

    # 4. Envoi vers GitHub
    # Changement du repertoire de travail
    os.chdir(f"/content/{nom_dossier_git}")

    print("Envoi vers GitHub en cours...")
    !git add "{nom_du_notebook}"  # On ajoute spécifiquement le notebook
    !git commit -m "Mise à jour du notebook de déploiement"
    !git push origin main # Ou 'master' selon votre branche

    print("Terminé ! Le notebook est maintenant sur le repo.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/parliament_speaker_diarisation/Notebook_Qui_Parle.ipynb'

Redemarrer session

In [ ]:
import sys
import types
import os
import torch
import torchaudio
import soundfile as sf
from google.colab import drive, userdata

print("1. Application des correctifs systèmes...")

# fonctions torchaudio
if not hasattr(torchaudio, "set_audio_backend"):
    torchaudio.set_audio_backend = lambda x: None
if not hasattr(torchaudio, "get_audio_backend"):
    torchaudio.get_audio_backend = lambda: "soundfile"
if not hasattr(torchaudio, "list_audio_backends"):
    torchaudio.list_audio_backends = lambda: ["soundfile"]

# Modules Torchaudio manquants
dummy_backend = types.ModuleType("torchaudio.backend")
dummy_common = types.ModuleType("torchaudio.backend.common")
try:
    dummy_common.AudioMetaData = torchaudio.AudioMetaData
except AttributeError:
    class AudioMetaData:
        def __init__(self, sr, nf, nc, bps, enc):
            self.sample_rate = sr
            self.num_frames = nf
            self.num_channels = nc
            self.bits_per_sample = bps
            self.encoding = enc
    dummy_common.AudioMetaData = AudioMetaData
sys.modules["torchaudio.backend"] = dummy_backend
sys.modules["torchaudio.backend.common"] = dummy_common

# On sauvegarde l'original une seule fois pour éviter la boucle infinie
if not hasattr(torch, "_original_load_backup"):
    torch._original_load_backup = torch.load

def permissive_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return torch._original_load_backup(*args, **kwargs)
torch.load = permissive_load

print("Correctifs appliqués. Le terrain est prêt.")

# import de pyannote
from pyannote.audio import Pipeline

# Connexion Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

FILE_PATH = "/content/drive/MyDrive/Qui_parle/2M_48k.wav"

if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Fichier introuvable : {FILE_PATH}")

print(f"Fichier trouvé : {FILE_PATH}")

try:
    TOKEN = userdata.get('HF_TOKEN')
except:
    TOKEN = "VOTRE_TOKEN_ICI"

MIN_SPEAKERS = 10
MAX_SPEAKERS = 15

print("\nChargement du pipeline...")
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=TOKEN
)
pipeline.to(torch.device("cuda"))

print("Lecture audio manuelle (SoundFile)...")
audio_data, sample_rate = sf.read(FILE_PATH)
waveform = torch.tensor(audio_data, dtype=torch.float32)

# Correction Mono/Stéréo
if waveform.dim() == 1:
    waveform = waveform.unsqueeze(0)
elif waveform.shape[1] < waveform.shape[0]:
    waveform = waveform.t()

waveform = waveform.to(torch.device("cuda"))

print("Démarrage de l'analyse...")
diarization = pipeline(
    {"waveform": waveform, "sample_rate": sample_rate},
    min_speakers=MIN_SPEAKERS,
    max_speakers=MAX_SPEAKERS
)

print("\n=== RÉSULTATS ===")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"Speaker {speaker}: {turn.start:.1f}s -> {turn.end:.1f}s")




Sauvegarde dans Drive

In [ ]:
# On définit le chemin de sauvegarde sur le Drive
OUTPUT_RTTM = "/content/drive/MyDrive/Qui_parle/resultats_diarisation.rttm"

print(f"\nSauvegarde des résultats dans : {OUTPUT_RTTM} ...")

# La fonction magique de Pyannote pour sauvegarder
with open(OUTPUT_RTTM, "w") as rttm_file:
    diarization.write_rttm(rttm_file)

print("Sauvegarde terminée ! En cas de crash, vous pourrez recharger ce fichier.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import os

# outils pour diarization
from pyannote.core import Annotation, Segment

# connexion drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

RTTM_PATH = "/content/drive/MyDrive/Qui_parle/resultats_diarisation.rttm"

if not os.path.exists(RTTM_PATH):
    print(f"ERREUR : Le fichier n'existe pas : {RTTM_PATH}")
    print("Avez-vous bien exécuté l'étape de sauvegarde précédente ?")
else:
    print(f"Fichier trouvé : {RTTM_PATH}")

    # recharger les données
    def charger_rttm_depuis_drive(chemin_fichier):
        """Lit un fichier RTTM et recrée un objet Annotation pour le graphique"""
        annotation = Annotation()

        with open(chemin_fichier, 'r') as f:
            for line in f:
                parts = line.strip().split()
                # Format standard RTTM :
                # SPEAKER file 1 start duration <NA> <NA> name <NA>
                if len(parts) >= 8 and parts[0] == "SPEAKER":
                    start = float(parts[3])
                    duration = float(parts[4])
                    speaker_label = parts[7]

                    # On ajoute le segment à l'annotation
                    annotation[Segment(start, start + duration)] = speaker_label

        return annotation

    # fonction de visualisation
    def visualiser_diarisation(diarization):
        # Calcul dynamique de la hauteur : 1 pouce par 3 speakers, minimum 6
        nb_speakers = len(diarization.labels())
        hauteur = max(6, nb_speakers * 0.5)

        fig, ax = plt.subplots(figsize=(20, hauteur))

        # On récupère tous les locuteurs uniques pour leur donner une couleur/hauteur
        speakers = sorted(diarization.labels()) # Trié pour être plus propre
        speaker_map = {label: i for i, label in enumerate(speakers)}

        # On trace chaque segment
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            idx = speaker_map[speaker]
            # On dessine une barre horizontale
            ax.hlines(idx, turn.start, turn.end, linewidth=30, colors=f"C{idx}")

        # Mise en forme
        ax.set_yticks(list(speaker_map.values()))
        ax.set_yticklabels(list(speaker_map.keys()))
        ax.set_xlabel("Temps (secondes)")
        ax.set_title("Timeline des Locuteurs (Chargée depuis le Drive)")
        ax.grid(True, axis='x', alpha=0.3)

        plt.tight_layout()
        plt.show()

    # Exécution
    print("Chargement des données...")
    # C'est ici que la magie opère : on recrée la variable 'diarization' depuis le fichier texte
    diarization_rechargee = charger_rttm_depuis_drive(RTTM_PATH)

    print("Génération du graphique...")
    visualiser_diarisation(diarization_rechargee)

In [ ]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    !pip install -qq matplotlib
    import matplotlib.pyplot as plt
import numpy as np

# fonction de vizualisation
def visualiser_diarisation(diarization):
    # Vous pouvez augmenter (20, 5) si vous avez beaucoup de locuteurs
    fig, ax = plt.subplots(figsize=(20, 6))

    # On récupère tous les locuteurs uniques
    speakers = diarization.labels()
    speaker_map = {label: i for i, label in enumerate(speakers)}

    # On trace chaque segment
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        idx = speaker_map[speaker]
        # On dessine une barre horizontale
        # 'C{idx}' utilise les couleurs par défaut de Matplotlib (C0, C1, etc.)
        ax.hlines(idx, turn.start, turn.end, linewidth=30, colors=f"C{idx}")

    # Mise en forme
    ax.set_yticks(list(speaker_map.values()))
    ax.set_yticklabels(list(speaker_map.keys()))
    ax.set_xlabel("Temps (secondes)")
    ax.set_title("Timeline des Locuteurs (Qui parle quand ?)")
    ax.grid(True, axis='x', alpha=0.3) # Grille verticale pour se repérer

    plt.tight_layout()
    plt.show()

# Lancement de la visualisation
# (Cela suppose que la variable 'diarization' existe déjà suite à l'étape précédente)
print("Génération du graphique...")
visualiser_diarisation(diarization)

import de whisper

In [ ]:
!pip install -qq git+https://github.com/openai/whisper.git soundfile torch torchaudio

Diarisation whisper

In [ ]:
import torch
import torchaudio.transforms as T
import soundfile as sf
import whisper
import os
from google.colab import drive

# 1. CONNEXION AU DRIVE
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. CHARGEMENT DU FICHIER AUDIO (Pour la découpe)
print("Lecture du fichier audio...")
FILE_PATH = "/content/drive/MyDrive/Qui_parle/2M_48k.wav"
RTTM_PATH = "/content/drive/MyDrive/Qui_parle/resultats_diarisation.rttm" # <-- Le fichier de secours

if not os.path.exists(FILE_PATH):
    print("ERREUR : Audio introuvable.")
elif not os.path.exists(RTTM_PATH):
    print(f"ERREUR : Le fichier de sauvegarde {RTTM_PATH} est introuvable.")
    print("La sauvegarde n'a peut-être pas eu le temps de se finir avant le crash ?")
else:
    # Lecture Audio
    audio_data, sample_rate = sf.read(FILE_PATH)
    waveform = torch.tensor(audio_data, dtype=torch.float32)
    if waveform.dim() == 1: waveform = waveform.unsqueeze(0)
    elif waveform.shape[1] < waveform.shape[0]: waveform = waveform.t()
    waveform = waveform.to("cuda")

    print(f"Audio chargé ({sample_rate} Hz).")
    print(f"Fichier de diarisation trouvé ({RTTM_PATH}).")

    # 3. VOS NOMS
    NOMS_REELS = {
        "SPEAKER_00": "Mme Estelle Youssouffa",
        "SPEAKER_01": "Mme Agnès Firmin Le Bodo",
        "SPEAKER_02": "M. Éric Coquerel",
        "SPEAKER_03": "M. Sébastien Lecornu",
        "SPEAKER_04": "M. Benjamin Lucas-Lundy",
        "SPEAKER_05": "M. Maxime Michelet",
        "SPEAKER_06": "Mme Marine Le Pen",
        "SPEAKER_07": "M. Jean-Victor Castor",
        "SPEAKER_08": "M. Nicolas Ray",
        "SPEAKER_09": "Inconnu",
        "SPEAKER_10": "Mme la présidente",
        "SPEAKER_11": "M. Laurent Baumel",
        "SPEAKER_12": "M. Nicolas Metzdorf",
        "SPEAKER_13": "M. Bruno Fuchs"
    }

    # 4. LECTURE DU FICHIER DE SAUVEGARDE (Parsing RTTM)
    segments_a_traiter = []
    with open(RTTM_PATH, 'r') as f:
        for line in f:
            parts = line.strip().split()
            # Format RTTM : SPEAKER file 1 start duration <NA> <NA> name <NA>
            if len(parts) >= 8 and parts[0] == "SPEAKER":
                start = float(parts[3])
                duration = float(parts[4])
                speaker = parts[7]
                end = start + duration
                segments_a_traiter.append((start, end, speaker))

    print(f"{len(segments_a_traiter)} segments récupérés depuis la sauvegarde.")

    # 5. TRANSCRIPTION (Whisper)
    print("\nChargement de Whisper et démarrage...")
    model = whisper.load_model("medium", device="cuda")
    resampler = T.Resample(orig_freq=sample_rate, new_freq=16000).to("cuda")

    full_transcript = []

    # On utilise la liste chargée depuis le fichier texte, plus la variable 'diarization'
    for start_sec, end_sec, speaker in segments_a_traiter:
        if end_sec - start_sec < 0.5: continue

        # Découpe
        start_f = int(start_sec * sample_rate)
        end_f = int(end_sec * sample_rate)
        chunk = waveform[:, start_f:end_f]

        # Traitement
        if chunk.shape[0] > 1: chunk = torch.mean(chunk, dim=0, keepdim=True)
        chunk_16k = resampler(chunk)

        # Texte
        res = model.transcribe(chunk_16k.squeeze().cpu().numpy(), language="fr", fp16=True)
        text = res['text'].strip()

        if text:
            nom = NOMS_REELS.get(speaker, speaker)
            ligne = f"[{start_sec:06.1f}s] {nom} : {text}"
            print(ligne)
            full_transcript.append(ligne)

    # Sauvegarde
    OUTPUT_TXT = "/content/drive/MyDrive/Qui_parle/TRANSCRIPTION_NOMMEE_REPRISE.txt"
    with open(OUTPUT_TXT, "w") as f:
        f.write("\n".join(full_transcript))
    print(f"\n🎉 Fini ! Sauvegardé dans {OUTPUT_TXT}")